Send Reports as HL7 v2

In [1]:
import requests
import os
from dotenv import load_dotenv
from prompt_toolkit.shortcuts import clear

load_dotenv()

server = os.getenv("V2_SERVER")
file_list = ['ORU_R01_R125.1_R0A.txt','ORU_R01_R125.1_RBS.txt','ORU_R01_R125.1_REP.txt','ORU_R01_R125.1_RR8.txt','ORU_R01_R125.1_RX1.txt']

for file in file_list:
    with open('Examples/' + file, 'rb') as f:
        r = requests.post(server, files={file: f})
        print(r.status_code)
        print(r.text)

200

MSA|CA|ORIE-251014-9|ISC|IGENE|MFT|202601281454||ACK^R01|ORIE-251014-9|T|2.3


Convert v2 examples to FHIR Messages, and then send to FHIR Server

In [5]:
from requests.auth import HTTPBasicAuth

url = os.getenv("OAUTH2_TOKEN")
clientId = os.getenv("CLIENT_ID")
clientSecret = os.getenv("CLIENT_SECRET")

the_data = {"grant_type": "client_credentials", "scope": "system/*.*"}
headers = {'Content-Type': 'application/x-www-form-urlencoded'}
response = requests.post(url, auth=HTTPBasicAuth(clientId, clientSecret),verify=False,data=the_data, headers=headers)
responseInclude = response.json()
token = responseInclude['access_token']

print(token)

/Users/kevinmayfield/Documents/GitHub/NHSNorthWestGMSA/Testing/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.1.20'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [6]:
server = os.getenv("V2_TOOLS")
fhirServer = os.getenv("FHIR_SERVER")
file_list = ['ORU_R01_R125.1_R0A.txt','ORU_R01_R125.1_RBS.txt','ORU_R01_R125.1_REP.txt','ORU_R01_R125.1_RR8.txt','ORU_R01_R125.1_RX1.txt']

for file in file_list:
    with open('Examples/' + file, 'rb') as f:
        r = requests.post(server, files={file: f})
        print(r.status_code)
        print(r.text)


200
{
  "resourceType": "Bundle",
  "entry": [
    {
      "fullUrl": "urn:uuid:02662277-86e7-4fd9-8014-a86bc8d80c73",
      "resource": {
        "resourceType": "MessageHeader",
        "destination": [
          {
            "endpoint": "https//fhir.mft.nhs.uk/Endpoint/EPIC",
            "receiver": {
              "identifier": {
                "system": "https://fhir.nhs.uk/Id/ods-organization-code",
                "value": "699X0"
              }
            }
          }
        ],
        "eventCoding": {
          "code": "R01",
          "system": "http://terminology.hl7.org/CodeSystem/v2-0003"
        },
        "sender": {
          "identifier": {
            "system": "https://fhir.nhs.uk/Id/ods-organization-code",
            "value": "699X0"
          }
        },
        "source": {
          "endpoint": "https://fhir.north-west.england.nhs.uk/Endpoint/IGENE",
          "software": "NW GLH"
        }
      }
    }
  ],
  "identifier": {
    "value": "ORIE-251014-9"
